In [ ]:
import keras
import numpy as np
import os
import PIL
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
from pylab import imread, imshow
from skimage.color import rgb2gray
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, core , UpSampling2D
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Dense, GlobalAveragePooling2D
#from fastai.models.unet import *
import keras.backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
import helpers

In [ ]:
def get_opt():
    #All parameters are defualt mentioned in paper
    return Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)



In [ ]:
def encoder_conv_block_batch_norm(inputs, n_filters, dropout, kernel_size=3, batchNorm = True):
    net = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(inputs)
    if batchNorm:
        net = BatchNormalization()(net)
    net = Activation("relu")(net)
        # second layer
    net = Dropout(dropout)(net)
    net = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal", padding="same")(net)
    if batchNorm:
        net = BatchNormalization()(net)
    net = Activation("relu")(net)
    return net

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        # second layer
        x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
        padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
    return x

In [ ]:
def get_unet_3(dropout, input_img):
    inputs = input_img
    patch_size = 32
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same',data_format='channels_last')(inputs)
    conv1 = Dropout(dropout)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same',data_format='channels_last')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    #
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same',data_format='channels_last')(pool1)
    conv2 = Dropout(dropout)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same',data_format='channels_last')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    #
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same',data_format='channels_last')(pool2)
    conv3 = Dropout(dropout)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same',data_format='channels_last')(conv3)

    up1 = UpSampling2D(size=(2, 2))(conv3)
    up1 = concatenate([conv2,up1])
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same',data_format='channels_last')(up1)
    conv4 = Dropout(dropout)(conv4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same',data_format='channels_last')(conv4)
    #
    up2 = UpSampling2D(size=(2, 2))(conv4)
    up2 = concatenate([conv1,up2])
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same',data_format='channels_last')(up2)
    conv5 = Dropout(dropout)(conv5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same',data_format='channels_last')(conv5)
    #
    conv6 = Conv2D(2, (1, 1), activation='relu',padding='same',data_format='channels_last')(conv5)
    conv6 = core.Reshape((2,32*32))(conv6)
    conv6 = core.Permute((2,1))(conv6)
    ############
    conv7 = core.Activation('softmax')(conv6)

    model = Model(inputs=inputs, outputs=conv7)
    opt  = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(optimizer = opt, loss = 'binary_crossentropy')
    return model

In [ ]:
def unet_gen(dropout, input_img, transposedConv, n_filters = 32):
    inputs = input_img
    patch_size = 32
    block1 = encoder_conv_block_batch_norm(inputs, n_filters =n_filters,
                                  kernel_size=3, dropout = dropout*0.5)
    pool1 = MaxPooling2D((2, 2))(block1)
    #conv1 = Conv2D(32, (3, 3), activation='relu', padding='same',data_format='channels_last')(inputs)
    #conv1 = Dropout(dropout)(conv1)
    #conv1 = Conv2D(32, (3, 3), activation='relu', padding='same',data_format='channels_last')(conv1)
    #pool1 = MaxPooling2D((2, 2))(conv1)
    #
    block2 = encoder_conv_block_batch_norm(pool1, n_filters = n_filters*2,
                                  kernel_size=3, dropout = dropout)
    pool2 = MaxPooling2D((2, 2))(block2) 
    #conv2 = Conv2D(64, (3, 3), activation='relu', padding='same',data_format='channels_last')(pool1)
    #conv2 = Dropout(dropout)(conv2)
    #conv2 = Conv2D(64, (3, 3), activation='relu', padding='same',data_format='channels_last')(conv2)
    #pool2 = MaxPooling2D((2, 2))(conv2)
    #
    bridge = encoder_conv_block_batch_norm(pool2, n_filters = n_filters*4,
                                  kernel_size=3, dropout = dropout)
    
    #conv3 = Conv2D(128, (3, 3), activation='relu', padding='same',data_format='channels_last')(pool2)
    #conv3 = Dropout(dropout)(conv3)
    #conv3 = Conv2D(128, (3, 3), activation='relu', padding='same',data_format='channels_last')(conv3)
    up1 = bridge
    if transposedConv:
        up1 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (up1)
    else:
        up1 = UpSampling2D(size=(2, 2))(up1)
    up1 = concatenate([block2,up1])
    up_block2 = encoder_conv_block_batch_norm(up1, n_filters = n_filters*2,
                                  kernel_size=3, dropout= dropout)
    
    #conv4 = Conv2D(64, (3, 3), activation='relu', padding='same',data_format='channels_last')(up1)
    #conv4 = Dropout(dropout)(conv4)
    #conv4 = Conv2D(64, (3, 3), activation='relu', padding='same',data_format='channels_last')(conv4)
    #
    up2 = up_block2
    if transposedConv:
        up2 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (up2)
    else:
        up2 = UpSampling2D(size=(2, 2))(up2)
    up2 = concatenate([block1,up2])
    up_block1 = encoder_conv_block_batch_norm(up2, n_filters = n_filters,
                                  kernel_size=3, dropout = dropout)
    #conv5 = Conv2D(32, (3, 3), activation='relu', padding='same',data_format='channels_last')(up2)
    #conv5 = Dropout(dropout)(conv5)
    #conv5 = Conv2D(32, (3, 3), activation='relu', padding='same',data_format='channels_last')(conv5)
    #
    outputs = Conv2D(1, (1, 1), activation='sigmoid',padding='same')(up_block1)
    #conv6 = core.Reshape((2,32*32))(conv6)
    #conv6 = core.Permute((2,1))(conv6)
    ############
    #conv7 = core.Activation('softmax')(conv6)
    opt = get_opt()
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer = opt, loss = 'binary_crossentropy')
    return model

In [ ]:
def get_transp_conv_unet(dropout, input_img):
    return unet_gen(dropout = dropout, input_img = input_img, transposedConv = True)

In [ ]:
def get_up_sample_unet(dropout, input_img):
    return unet_gen(dropout = dropout, input_img = input_img, transposedConv = False)

In [ ]:
def get_unet_seismic(input_img, dropout, n_filters=16, batchnorm=True):
    # contracting path
    print("SEISMIC UNET")
    c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout)(p1)
    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dropout)(p2)
    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dropout)(p3)
    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dropout)(p4)
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    # expansive path
    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    opt  = get_opt()
    model.compile(optimizer = opt, loss = 'binary_crossentropy')
    model.summary()
    return model

In [ ]:
def load_images_from_folder(folder_path):
    file_names = os.listdir(folder_path)
    file_names.sort()
    images = []
    for filename in file_names:
        try:
            image =imread(folder_path + filename)
            images.append(rgb2gray(image).reshape(32,32,1))
        except:
            continue
    return images

In [ ]:
def generator(x_train, y_train, batch_size):
    data_gen = ImageDataGenerator(
        horizontal_flip = True, vertical_flip = True,
    ).flow(x_train, x_train, batch_size, seed=1)
    mask_gen = ImageDataGenerator(
        horizontal_flip = True, vertical_flip = True
    ).flow(y_train, y_train, batch_size, seed=1)
    while True:
        x_batch, _ = data_gen.next()
        y_batch, _ = mask_gen.next()
        yield x_batch, y_batch

## Test data augmentation function

In [ ]:

X_train = np.array(load_images_from_folder("../../data/DRIVE/training/patches/"))
y_train = np.array(load_images_from_folder("../../data/DRIVE/training/patchLabels/"))
image_batch, mask_batch = next(generator(X_train, y_train, 8))
fix, ax = plt.subplots(8,2, figsize=(8,20))
for i in range(8):
    ax[i,0].imshow(image_batch[i,:,:,0])
    ax[i,1].imshow(mask_batch[i,:,:,0])
plt.show()

In [ ]:
def train(epochs, n_folds, model_dict):
    X = np.array(load_images_from_folder("../../data/DRIVE/training/patches/"))
    y = np.array(load_images_from_folder("../../data/DRIVE/training/patchLabels/"))
    load_model = model_dict['load_model']
    args = model_dict['args']
    model = load_model(**args)
    batch_size = 32
    historyList =[]
    if n_folds > 0:
        if (len(X) % n_folds) != 0:
            raise Exception('Same eye in validation and traning set, Patches: ', X_train.shape[1], ' Folds', n_folds)
        sizeFold = len(X) // n_folds
        fold_idx = []
        for fold in range(n_folds):
            for curIdx in range(sizeFold):
                fold_idx.append(fold)
        fold_idx = np.array(fold_idx)
    
        for fold in range(n_folds):
            train_idx = fold_idx != fold
            test_idx = fold_idx == fold
            X_train = X[train_idx]
            y_train = y[train_idx]
            X_test = X[test_idx]
            y_test = y[test_idx]
            history = model.fit_generator(generator(X_train, y_train, batch_size = batch_size),
                                  epochs = epochs, steps_per_epoch = len(X_train)//batch_size,
                                         validation_data = (X_test,y_test))
            historyList.append(history)
    else : 
        print(X.shape)
        print(y.shape)
        
        history = model.fit_generator(generator(X, y, batch_size = batch_size),
                                      epochs = epochs, steps_per_epoch = len(X)//batch_size)
        historyList.append(history)
    return model, historyList
    

## Train model on the full data set, no cross validation

In [330]:
img = patches[0].reshape(1,32,32,1)
print(img.shape)
args1 = {'input_img' : img}
model_dict = {'load_model' : transfer_learning, 'args' : args1}
res, historyList = train(epochs = 1, n_folds = 0, model_dict = model_dict)


(1, 32, 32, 1)


ValueError: The input must have 3 channels; got `input_shape=(32, 32, 1)`

## Compare n models with different parameters

In [ ]:
epochs = 40
args1 = {'input_img' : Input((32,32,1)), 'dropout' : 0.05}
args2 = {'input_img' : Input((32,32,1)), 'dropout' : 0.2}
args3 = {'input_img' : Input((32,32,1)), 'dropout' : 0.3}

model_dict1 = {'load_model' : get_transp_conv_unet, 'args' : args1, 'name': 'TranspConv0.05'}
model_dict2 = {'load_model' : get_unet_seismic, 'args' : args1, 'name': 'Seismic'}
model_dict3 = {'load_model' : get_up_sample_unet, 'args' : args1, 'name': 'UpSample'}
model_dict4 = {'load_model' : get_transp_conv_unet, 'args' : args2, 'name': 'TranspConv0.2'}
model_dict5 = {'load_model' : get_transp_conv_unet, 'args' : args3, 'name': 'TranspConv0.3'}
model_dict6 = {'load_model' : get_unet_seismic, 'args' : args2, 'name': 'Seismic0.2'}



model_dicts = [model_dict1, model_dict2, model_dict3]
models = []
historyLists =[]
for model_dict in model_dicts:
    res, historyList = train(epochs = epochs, n_folds = 5, model_dict = model_dict)
    models.append(res)
    historyLists.append(historyList)

In [ ]:
full_model_history_df = pd.DataFrame()
for cur_model_nr in range(len(models)):
    loss_data = ([i.history['loss'] for i in historyLists[cur_model_nr]])
    val_loss_data = ([i.history['val_loss'] for i in historyLists[cur_model_nr]])
    all_data = loss_data + val_loss_data
    all_data.append( [i+1 for i in range(epochs)])
    model_history_df = pd.DataFrame(all_data).T
    column_names = []
    foldLossName = []
    for i in range(len(loss_data)):
        foldLossName.append("fold"+str(i)+"_loss")
    column_names += foldLossName
    foldValLossName = []
    for i in range(len(val_loss_data)):
        foldValLossName.append("fold"+str(i)+"val_loss")
    column_names += foldValLossName
    column_names.append('epochs')
    model_history_df.columns=column_names
    model_history_df["model"] = model_dicts[cur_model_nr]['name']
    #plt.plot(history.history['loss'])
    #plt.plot(history.history['val_loss'])
    #plt.xlabel("epoch")
    #plt.ylabel("loss")
    #plt.show()
    model_history_df = pd.melt(model_history_df, value_vars=foldLossName+ foldValLossName,
                               id_vars = ['epochs','model'] ,value_name = "loss")
    types = ['train', 'validation']
    trainOrValidation = []
    for i in types:
        for j in range(model_history_df.shape[0]//2):
            trainOrValidation.append(i)
    model_history_df['type'] =trainOrValidation
    model_history_df = model_history_df.astype({'epochs': int})
    full_model_history_df = pd.concat([full_model_history_df, model_history_df], ignore_index = True)
print(full_model_history_df)

In [ ]:
plt.style.use('ggplot')
ax = sns.lineplot(x="epochs", y="loss", style="type", hue= 'model', data=full_model_history_df)
legend = ax.legend()
legend.texts[0].set_text("")
plt.show()


In [ ]:
nr_of_images = 10
fig, ax = plt.subplots(nr_of_images,4, figsize=(8,20))
fig.tight_layout()
for image_nr in range(nr_of_images):
    ax[image_nr,0].imshow(X_train[image_nr].reshape(32,32))
    ax[image_nr,1].imshow(y_train[image_nr].reshape(32,32))
    y_pred = res.predict(X_train[image_nr:(image_nr+1)]).reshape(32,32)
    y_pred_thr = y_pred.copy()
    y_pred_thr[y_pred > 0.5] = 1
    y_pred_thr[y_pred <= 0.5] = 0
    ax[image_nr,2].imshow(y_pred)
    ax[image_nr,3].imshow(y_pred_thr)
    for j in range(4):
        ax[image_nr,j].set_xticklabels([])
        ax[image_nr,j].set_yticklabels([])
plt.show()



In [ ]:
def segment_image(image, mask, model, m, dataset):
    image = rgb2gray(image)
    patches, indexes = helpers.image_to_non_overlapping_patches(image, mask, m, helpers.Dataset.DRIVE)
    patches = np.array(patches)
    patches = patches.reshape(patches.shape[0], m, m, 1)
    y_pred = model.predict(patches).reshape(patches.shape[0], 32,32)
    y_pred_thr = y_pred.copy()
    y_pred_thr[y_pred > 0.5] = 1
    y_pred_thr[y_pred <= 0.5] = 0
    image = helpers.patches_to_image(y_pred_thr, indexes, m, image.shape[0], image.shape[1] )
    return image

In [ ]:
drive_images = helpers.load_images_from_folder(helpers.DRIVE_TEST_IMAGES_PATH)
drive_images_color = load_images_from_folder_color(helpers.DRIVE_TEST_IMAGES_PATH)
drive_mask = helpers.load_images_from_folder(helpers.DRIVE_TEST_MASK_PATH)
drive_segmented = helpers.load_images_from_folder(helpers.DRIVE_TEST_SEG_1_PATH)

In [ ]:
image_segmentation = segment_image(drive_images[0], drive_mask[0],transfer_model , 32, helpers.Dataset.DRIVE)
fig, ax = plt.subplots(1,3, figsize=(20,20))
ax[0].imshow(image_segmentation)
ax[1].imshow(drive_segmented[0])
ax[2].imshow(helpers.histogram_equalization(drive_images[0]))

plt.show()

In [ ]:
def transfer_learning(input_img):
    import ssl
    ssl._create_default_https_context = ssl._create_unverified_context
   
    from keras.applications.vgg16 import VGG16 as PTModel
    base_pretrained_model = PTModel(input_shape = input_img.shape[1:], include_top = False, weights = 'imagenet')
    base_pretrained_model.trainable = False
    # base_pretrained_model.summary()

    from collections import defaultdict, OrderedDict
    from keras.models import Model
    layer_size_dict = defaultdict(list)
    inputs = []
    for lay_idx, c_layer in enumerate(base_pretrained_model.layers):
        if not c_layer.__class__.__name__ == 'InputLayer':
            layer_size_dict[c_layer.get_output_shape_at(0)[1:3]] += [c_layer]
        else:
            inputs += [c_layer]
    # freeze dict
    layer_size_dict = OrderedDict(layer_size_dict.items())
    for k,v in layer_size_dict.items():
        print(k, [w.__class__.__name__ for w in v])
    
    pretrained_encoder = Model(inputs = base_pretrained_model.get_input_at(0), 
                           outputs = [v[-1].get_output_at(0) for k, v in layer_size_dict.items()])
    pretrained_encoder.trainable = False
    n_outputs = pretrained_encoder.predict([input_img])
    for c_out, (k, v) in zip(n_outputs, layer_size_dict.items()):
        print(c_out.shape, 'expected', k)
    
    
    from keras.layers import Input, Conv2D, concatenate, UpSampling2D, BatchNormalization, Activation, Cropping2D, ZeroPadding2D
    x_wid, y_wid = input_img.shape[1:3]
    in_t0 = Input(input_img.shape[1:], name = 'T0_Image')
    wrap_encoder = lambda i_layer: {k: v for k, v in zip(layer_size_dict.keys(), pretrained_encoder(i_layer))}

    t0_outputs = wrap_encoder(in_t0)
    lay_dims = sorted(t0_outputs.keys(), key = lambda x: x[0])
    skip_layers = 2
    last_layer = None
    for k in lay_dims[skip_layers:]:
        cur_layer = t0_outputs[k]
        channel_count = cur_layer._keras_shape[-1]
        cur_layer = Conv2D(channel_count//2, kernel_size=(3,3), padding = 'same', activation = 'linear')(cur_layer)
        cur_layer = BatchNormalization()(cur_layer) # gotta keep an eye on that internal covariant shift
        cur_layer = Activation('relu')(cur_layer)

        if last_layer is None:
            x = cur_layer
        #else:
         #   last_channel_count = last_layer._keras_shape[-1]
         #   x = Conv2D(last_channel_count//2, kernel_size=(3,3), padding = 'same')(last_layer)
         #   x = UpSampling2D((2, 2))(x)
         #   x = concatenate([cur_layer, x])
        last_layer = x
    final_output = Conv2D(input_img.shape[-1], kernel_size=(1,1), padding = 'same', activation = 'sigmoid')(last_layer)
    crop_size = 20
    final_output = Cropping2D((crop_size, crop_size))(final_output)
    final_output = ZeroPadding2D((crop_size, crop_size))(final_output)
    unet_model = Model(inputs = [in_t0],
                      outputs = [final_output])
    unet_model.summary()
    
    unet_model.compile(optimizer=Adam(1e-3, decay = 1e-6), 
                       loss=dice_p_bce, 
                       metrics=[dice_coef, 'binary_accuracy', true_positive_rate])
    
    return unet_model
    



In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
        intersection = K.sum(y_true * y_pred, axis=[1,2,3])
        union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
        return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

In [ ]:
img = np.resize(drive_images[0], 32*32*3)
transfer_learning(img.reshape(1,32,32,3))


In [ ]:
def load_images_from_folder_color(folder_path):
    file_names = os.listdir(folder_path)
    file_names.sort()
    images = []
    for filename in file_names:
        try:
            image =imread(folder_path + filename)
            images.append(image)
        except:
            continue
    print(images)
    return images


In [ ]:
def dice_p_bce(in_gt, in_pred):
        return 0.0*binary_crossentropy(in_gt, in_pred) - dice_coef(in_gt, in_pred)

In [ ]:
def true_positive_rate(y_true, y_pred):
        return K.sum(K.flatten(y_true)*K.flatten(K.round(y_pred)))/K.sum(y_true)

In [325]:
patches, labels = helpers.get_image_pathes(drive_images[0], drive_segmented[0], 32, 100, helpers.Dataset.DRIVE, mask=drive_mask[0])